#11.1 Audio Extraction

In [ ]:
import subprocess

def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


extract_audio_from_video(
    "./files/podcast.mp4",
    "./files/podcast.mp3",
)

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.0.13.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

#11.2 Cutting The Audio

In [1]:
import subprocess
from pydub import AudioSegment


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


track = AudioSegment.from_mp3("./files/podcast.mp3")

ten_minutes = 1 * 60 * 1000

In [2]:
import math

chunks = math.ceil(len(track) / ten_minutes)

for i in range(chunks):
    start_time = i * ten_minutes
    end_time = (i + 1) * ten_minutes

    chunk = track[start_time:end_time]

    chunk.export(f"./files/chunks/chunk_{i}.mp3", format="mp3")

#11.3 Whisper Transcript 

In [ ]:
import subprocess
from pydub import AudioSegment
import math


def extract_audio_from_video(video_path, audio_path):
    command = [
        "ffmpeg",
        "-i",
        video_path,
        "-vn",
        audio_path,
    ]
    subprocess.run(command)


def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len

        chunk = track[start_time:end_time]

        chunk.export(f"./{chunks_folder}/chunk_{i}.mp3", format="mp3")

In [4]:
import openai
import glob


def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    final_transcript = ""
    for file in files:
        with open(file, "rb") as audio_file:
            # transcript = openai.Audio.transcribe(
            #     "whisper-1",
            #     audio_file,
            # )
            transcript = openai.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                language="ko"
            )
            final_transcript += transcript.text
            # final_transcript += transcript["text"]
    with open(destination, "w") as file:
        file.write(final_transcript)


transcribe_chunks("./files/chunks", "./files/transcript.txt")